<a href="https://colab.research.google.com/github/LaizaCostaa/previsao_preco_imoveis/blob/main/previsao_precos_imoveis_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


** ANÁLISE DE PREÇO DE IMÓVEIS PARTE II **

Uma análise preditiva dos preços das casas, cuja análise exploratória está neste link: https://colab.research.google.com/drive/1UhBuudCS7fvnb5IUFOa4TLcMv-JuPMGe?usp=sharing


Decidi dividir em duas partes porque queria fazer uma análise exploratória mais detalhada e não deixar o código tão poluído com análise e aplicação do modelo ao mesmo tempo.


* Artigo da análise exploratória: https://medium.com/@laizamare/an%C3%A1lise-de-oportunidades-e-estrat%C3%A9gias-de-neg%C3%B3cio-no-setor-imobili%C3%A1rio-estudo-de-caso-house-d629e2378e78


* Artigo com análise preditiva


In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


In [4]:

# ler o csv
df = pd.read_csv("/content/df.csv")
display(df)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,renovated,price group,month,year,location
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340.0,5650.0,0.0,Group 1,10.0,2014.0,Seattle
1,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,1340.0,5650.0,0.0,Group 1,10.0,2014.0,Tukwila
2,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190.0,6780.0,0.0,Group 1,6.0,2014.0,Seattle
3,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,1190.0,6780.0,0.0,Group 1,6.0,2014.0,Tukwila
4,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,98178,47.5091,-122.244,2660.0,8712.0,0.0,Group 2,7.0,2014.0,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,8945100320,2014-10-08,224097.0,3,1.50,1420,8580,1.0,0,0,...,98023,47.3076,-122.362,1200.0,8580.0,0.0,Group 1,10.0,2014.0,Auburn
15728,8945100320,2014-10-08,224097.0,3,1.50,1420,8580,1.0,0,0,...,98023,47.3076,-122.362,1200.0,8580.0,0.0,Group 1,10.0,2014.0,Federal Way
15729,8731900640,2015-03-24,315000.0,3,1.75,2380,10450,1.5,0,0,...,98023,47.3143,-122.370,2320.0,8500.0,0.0,Group 2,3.0,2015.0,Auburn
15730,8731900640,2015-03-24,315000.0,3,1.75,2380,10450,1.5,0,0,...,98023,47.3143,-122.370,2320.0,8500.0,0.0,Group 2,3.0,2015.0,Federal Way


**PRÉ-PROCESSAMENTO DOS DADOS**

In [57]:
df_pre_tratado = df.drop(['date', 'price group', 'month', 'year', 'renovated'], axis=1)

In [58]:
one_hot = pd.get_dummies(df_pre_tratado['location'])

In [59]:
one_hot

,Algona,Auburn,Bellevue,Bothell,Burien,Des Moines,Duvall,Federal Way,Issaquah,Kenmore,...,Mercer Island,Newcastle,Normandy Park,Redmond,Renton,Sammamish,Seatac,Seattle,Shoreline,Tukwila
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15728,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
15729,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15730,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
df_tratado = df_pre_tratado.drop("location", axis=1)
df_tratado = df_tratado.join(one_hot)
df_tratado


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,Mercer Island,Newcastle,Normandy Park,Redmond,Renton,Sammamish,Seatac,Seattle,Shoreline,Tukwila
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,0,0,0,0,0,0,0,1,0,0
1,7129300520,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
2,4060000240,205425.0,2,1.00,880,6780,1.0,0,0,4,...,0,0,0,0,0,0,0,1,0,0
3,4060000240,205425.0,2,1.00,880,6780,1.0,0,0,4,...,0,0,0,0,0,0,0,0,0,1
4,4058801670,445000.0,3,2.25,2100,8201,1.0,0,2,3,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,8945100320,224097.0,3,1.50,1420,8580,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
15728,8945100320,224097.0,3,1.50,1420,8580,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
15729,8731900640,315000.0,3,1.75,2380,10450,1.5,0,0,3,...,0,0,0,0,0,0,0,0,0,0
15730,8731900640,315000.0,3,1.75,2380,10450,1.5,0,0,3,...,0,0,0,0,0,0,0,0,0,0


**DEFININDO DADOS DE TREINO E DADOS DE TESTE**

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
y = df_tratado['price']
x = df_tratado.loc[:, df_tratado.columns != "price"]

In [63]:
x

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Mercer Island,Newcastle,Normandy Park,Redmond,Renton,Sammamish,Seatac,Seattle,Shoreline,Tukwila
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,0,1,0,0
1,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,1
2,4060000240,2,1.00,880,6780,1.0,0,0,4,6,...,0,0,0,0,0,0,0,1,0,0
3,4060000240,2,1.00,880,6780,1.0,0,0,4,6,...,0,0,0,0,0,0,0,0,0,1
4,4058801670,3,2.25,2100,8201,1.0,0,2,3,8,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15727,8945100320,3,1.50,1420,8580,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
15728,8945100320,3,1.50,1420,8580,1.0,0,0,3,6,...,0,0,0,0,0,0,0,0,0,0
15729,8731900640,3,1.75,2380,10450,1.5,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
15730,8731900640,3,1.75,2380,10450,1.5,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [64]:
y

0        221900.0
1        221900.0
2        205425.0
3        205425.0
4        445000.0
           ...   
15727    224097.0
15728    224097.0
15729    315000.0
15730    315000.0
15731    218000.0
Name: price, Length: 15732, dtype: float64

In [65]:
# Separando dados treino e teste

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [66]:
x_train

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,Mercer Island,Newcastle,Normandy Park,Redmond,Renton,Sammamish,Seatac,Seattle,Shoreline,Tukwila
8044,1887500045,4,2.00,2460,5921,1.0,0,0,4,7,...,0,0,0,0,0,0,0,0,0,0
11503,710600070,4,3.50,2650,3127,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
12910,6143600580,3,1.75,1490,10125,1.0,0,0,4,7,...,0,0,0,0,0,0,0,0,0,0
10093,3022900070,3,2.00,2360,6145,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
6266,1868903130,4,1.00,1540,5000,1.5,0,0,4,7,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,1922059027,4,1.00,1450,32234,1.5,0,0,4,6,...,0,0,0,0,0,0,0,0,0,0
30,2171400197,5,3.00,2520,5500,1.0,0,0,3,8,...,0,0,0,0,0,0,0,1,0,0
7302,2133020020,3,2.50,1920,15260,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
3325,7212650200,3,2.50,2180,15484,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0


In [67]:
y_train

8044     247500.0
11503    674950.0
12910    184000.0
10093    348000.0
6266     542000.0
           ...   
9931     282510.0
30       350000.0
7302     372000.0
3325     350000.0
9367     880000.0
Name: price, Length: 11012, dtype: float64

**APLICAÇÃO DO MODELO**

In [68]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [69]:
# imputer para preencher os valores ausentes com a média
imputer = SimpleImputer(strategy='mean')

x_train_imputed = imputer.fit_transform(x_train)

In [71]:

rf_reg = RandomForestRegressor()
rf_reg.fit(x_train_imputed, y_train)

RandomForestRegressor()

In [72]:
RandomForestRegressor()

RandomForestRegressor()

In [73]:
#verificando os parâmetros de determinado imóvel nos dados de teste

y_test.iloc[0]

633100.0

In [74]:
x_test.iloc[0].head(10)

id             205000450.0
bedrooms               4.0
bathrooms              2.5
sqft_living         2470.0
sqft_lot           33305.0
floors                 2.0
waterfront             0.0
view                   0.0
condition              3.0
grade                  9.0
Name: 2888, dtype: float64

In [75]:
# Verificando o valor sugerido pelo modelo considerando os parâmetros dos dados de teste

rf_reg.predict(x_test.iloc[0].values.reshape(1, -1))

array([623469.7])

* Valor bem próximo do encontrado nos dados de teste

In [78]:
# Verificando parâmetros de imóveis específicos


n = 1

print(y_test.iloc[n])
print(x_test.iloc[n].head(10))

rf_reg.predict(x_test.iloc[n].values.reshape(1, -1))

197000.0
id             6.303400e+09
bedrooms       2.000000e+00
bathrooms      1.000000e+00
sqft_living    7.700000e+02
sqft_lot       8.636000e+03
floors         1.000000e+00
waterfront     0.000000e+00
view           0.000000e+00
condition      2.000000e+00
grade          6.000000e+00
Name: 5166, dtype: float64


array([195115.53])



*  O Imóvel encontrado está no valore de U$197.000 dólares e foi avaliado pelo modelo em U$195.115, ou seja, muito próximo do valor real.
*   O imóvel atual tem menos quartos  do que o anterior, uma correlação que se alinha muito bem com a nossa análise exploratória.

**AVALIAÇÃO**

In [82]:
rf_reg.score(x_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


0.9034438544441875

* O modelo atingiu uma acurácia 90.34%